### please install binance by: pip install python-binance 

In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date,datetime
from binance import Client, ThreadedWebsocketManager,ThreadedDepthCacheManager

In [14]:
# Little function to keep track of code execution.
def Keep_Track():
    print("Executed successfully. " + datetime.now().strftime("%a %d %b @ %H:%M:%S") + "\n")

In [15]:
def get_historical_klines_binance(client,ticker,start_time,end_time,kline_interval,path=None):
    '''
    function to download historical kline data from binance
    client:binance client object
    ticker: the ccy ticker eg: BTCBUSD
    start_time: start time in string format e.g '1 Jan 2021'
    end_time: end time in string format
    kline_interval: kline interval eg: 15MINUTE
    save_file: if true will save in feather format
    '''
    #query binance API and format returned dictionary into dataframe
    raw = client.get_historical_klines(symbol=ticker, interval=getattr(Client,f'KLINE_INTERVAL_{kline_interval}'),start_str=start_time,end_str=end_time)
    df=pd.DataFrame(raw)
    df.columns=['Open Time','Open','High','Low','Close','Volume','Close Time','Quote Asset Volume','Number of Trades','TB Base Volume','TB Quote Volume','ignore']
    df['Open Time']=pd.to_datetime(df['Open Time']/1000,unit='s')
    df['Close Time']=pd.to_datetime(df['Close Time']/1000,unit='s')
    # change object values into numeric 
    numeric_columns=['Open','High','Low','Close','Volume','Quote Asset Volume','TB Base Volume','TB Quote Volume']
    df[numeric_columns]=df[numeric_columns].apply(pd.to_numeric,axis=1)
    #save file into feather
    if path is not None:
        file_name = f'{ticker}_{kline_interval}.feather'
        df.to_feather(os.path.join(path,file_name))
        print(f'successful saved {file_name}')

    print(f'total {len(df)} rows')
    print(f"first date: {np.min(df['Open Time'])}")
    print(f"last date: {np.max(df['Close Time'])}")
    Keep_Track()
    return df

In [18]:
client = Client('','')
start_time='1 Jan 2022'
end_time='6 May 2022'
ticker = 'BTCUSDT'
kline_interval ='1HOUR'
path='../data'

In [19]:
df = get_historical_klines_binance(client,ticker,start_time,end_time,kline_interval)
df

total 3000 rows
first date: 2022-01-01 00:00:00
last date: 2022-05-05 23:59:59.999000072
Executed successfully. Sun 29 May @ 21:07:24



,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,ignore
0,2022-01-01 00:00:00,46216.93,46731.39,46208.37,46656.13,1503.33095,2022-01-01 00:59:59.999000072,6.987999e+07,38608,806.06536,3.746216e+07,0
1,2022-01-01 01:00:00,46656.14,46949.99,46574.06,46778.14,943.81539,2022-01-01 01:59:59.999000072,4.412715e+07,31872,491.18067,2.296327e+07,0
2,2022-01-01 02:00:00,46778.14,46928.94,46721.96,46811.77,485.16860,2022-01-01 02:59:59.999000072,2.272067e+07,24364,206.35113,9.662607e+06,0
3,2022-01-01 03:00:00,46811.77,46916.63,46760.12,46813.20,562.88971,2022-01-01 03:59:59.999000072,2.636326e+07,19882,248.28212,1.162938e+07,0
4,2022-01-01 04:00:00,46813.21,46887.33,46591.23,46711.05,861.88389,2022-01-01 04:59:59.999000072,4.027204e+07,23357,397.53563,1.857540e+07,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2022-05-05 19:00:00,36386.83,36390.00,35571.90,36316.00,10383.49886,2022-05-05 19:59:59.999000072,3.739403e+08,198037,4819.59859,1.735694e+08,0
2996,2022-05-05 20:00:00,36316.01,36477.71,36132.85,36466.44,3130.20550,2022-05-05 20:59:59.999000072,1.135683e+08,57342,1669.02433,6.055295e+07,0
2997,2022-05-05 21:00:00,36466.44,36600.00,36328.80,36458.58,1797.64214,2022-05-05 21:59:59.999000072,6.557188e+07,43290,879.34960,3.207609e+07,0
2998,2022-05-05 22:00:00,36458.58,36495.00,36211.89,36238.63,1455.11242,2022-05-05 22:59:59.999000072,5.294698e+07,31769,728.06372,2.649489e+07,0


In [21]:
top_mkt = pd.read_excel('../TopMarket_Cap.xlsx')
tickers = list(top_mkt.symbol+'USDT')

In [25]:
client = Client(apikey,secret)
start_time='1 Jan 2022'
end_time='6 May 2022'
kline_interval ='1HOUR'
path='../data'

for tkrs in tickers:
    try:
        df = get_historical_klines_binance(client,tkrs,start_time,end_time,kline_interval,path)
    except:
        print(f'!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!{tkrs} download error!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')

successful saved BTCUSDT_1HOUR.feather
total 3000 rows
first date: 2022-01-01 00:00:00
last date: 2022-05-05 23:59:59.999000072
Executed successfully. Sun 29 May @ 21:09:12

successful saved ETHUSDT_1HOUR.feather
total 3000 rows
first date: 2022-01-01 00:00:00
last date: 2022-05-05 23:59:59.999000072
Executed successfully. Sun 29 May @ 21:09:13

successful saved BNBUSDT_1HOUR.feather
total 3000 rows
first date: 2022-01-01 00:00:00
last date: 2022-05-05 23:59:59.999000072
Executed successfully. Sun 29 May @ 21:09:13

successful saved XRPUSDT_1HOUR.feather
total 3000 rows
first date: 2022-01-01 00:00:00
last date: 2022-05-05 23:59:59.999000072
Executed successfully. Sun 29 May @ 21:09:14

successful saved ADAUSDT_1HOUR.feather
total 3000 rows
first date: 2022-01-01 00:00:00
last date: 2022-05-05 23:59:59.999000072
Executed successfully. Sun 29 May @ 21:09:15

successful saved SOLUSDT_1HOUR.feather
total 3000 rows
first date: 2022-01-01 00:00:00
last date: 2022-05-05 23:59:59.999000072
Ex